In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error

In [72]:
class MyNeuralNetwork:            
    def __init__(self, layers, learning_rate, momentum, activation, validation_percentage=0):
        self.L = len(layers)
        self.n = layers.copy()
        self.xi = [np.zeros(l) for l in layers]
        self.w = [np.random.randn(layers[i], layers[i-1]) for i in range(1, self.L)]
        self.theta = [np.zeros(l) for l in layers]
        self.delta = [np.zeros(l) for l in layers]
        self.d_w = [np.zeros_like(w) for w in self.w]
        self.d_theta = [np.zeros_like(t) for t in self.theta]
        self.d_w_prev = [np.zeros_like(w) for w in self.w]
        self.d_theta_prev = [np.zeros_like(t) for t in self.theta]
        self.learning_rate = learning_rate 
        self.momentum = momentum 
        self.activation = activation
        self.validation_percentage = validation_percentage
        self.loss_epochs = []

    def activation_function(self, x):
        if self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        elif self.activation == 'relu':
            return np.maximum(0, x)
        elif self.activation == 'linear':
            return x
        elif self.activation == 'tanh':
            return np.tanh(x)
        else:
            raise ValueError("Invalid activation function")

    def activation_derivative(self, x):
        if self.activation == 'sigmoid':
            return x * (1 - x)
        elif self.activation == 'relu':
            return np.where(x > 0, 1, 0)
        elif self.activation == 'linear':
            return 1
        elif self.activation == 'tanh':
            return 1 - x**2
        else:
            raise ValueError("Invalid activation function")

    def forward_pass(self, x):
        self.xi[0] = x
        for i in range(1, self.L):
            self.xi[i] = self.activation_function(np.dot(self.w[i-1], self.xi[i-1]) - self.theta[i])

    def backward_pass(self, y):
        self.delta[-1] = (self.xi[-1] - y) * self.activation_derivative(self.xi[-1])
        for i in range(self.L - 2, 0, -1):
            self.delta[i] = np.dot(self.w[i].T, self.delta[i+1]) * self.activation_derivative(self.xi[i])

    def update_weights(self):
        for i in range(1, self.L):
            self.d_w[i-1] = self.learning_rate * np.outer(self.delta[i], self.xi[i-1]) + self.momentum * self.d_w_prev[i-1]
            self.d_theta[i] = self.learning_rate * self.delta[i] + self.momentum * self.d_theta_prev[i]
            self.w[i-1] -= self.d_w[i-1]
            self.theta[i] -= self.d_theta[i]
            self.d_w_prev[i-1] = self.d_w[i-1]
            self.d_theta_prev[i] = self.d_theta[i]
            
    def mape(self, y_true, y_pred):
        mask = y_true != 0
        return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

    def fit(self, X, y, epochs, batch_size=32, learning_rate_decay=0.1):
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=self.validation_percentage, random_state=42)

        for epoch in range(epochs):
            for _ in range(0, len(X_train), batch_size):
                batch_indices = np.random.choice(len(X_train), min(batch_size, len(X_train)), replace=False)
                if len(batch_indices) < 2:  # Skip batches with less than 2 samples
                    continue

                X_batch, y_batch = X_train[batch_indices], y_train[batch_indices]

                for i in range(len(X_batch)):
                    x_sample, y_sample = X_batch[i], y_batch[i]
                    self.forward_pass(x_sample)
                    self.backward_pass(y_sample)
                    self.update_weights()

            # Calculate metrics with safeguards
            train_predictions = self.predict(X_train)
            val_predictions = self.predict(X_val)

            training_error = mean_squared_error(y_train, train_predictions)
            validation_error = mean_squared_error(y_val, val_predictions)

            training_r2 = r2_score(y_train, train_predictions) if len(y_train) > 1 else None
            validation_r2 = r2_score(y_val, val_predictions) if len(y_val) > 1 else None

            print(f"Epoch {epoch}/{epochs} - Training Error: {training_error}, Validation Error: {validation_error}, Training R2: {training_r2}, Validation R2: {validation_r2}")

            # Learning rate decay
            self.learning_rate *= (1.0 / (1.0 + learning_rate_decay * epoch))

        final_train_mape = self.mape(y_train, self.predict(X_train))
        final_val_mape = self.mape(y_val, self.predict(X_val))
        final_train_r2 = r2_score(y_train, self.predict(X_train))
        final_val_r2 = r2_score(y_val, self.predict(X_val))
        print(f"Final Training MAPE: {final_train_mape}, Final Validation MAPE: {final_val_mape}")
        print(f"Final Training R2: {final_train_r2}")
        print(f"Final Training MSE: {training_error}, Final Validation MSE: {validation_error}")
        return np.array(self.loss_epochs)
    
    
    def predict(self, X):
        predictions = []
        for i in range(len(X)):
            self.forward_pass(X[i])
            predictions.append(self.xi[-1][0])
        return np.array(predictions)

    def get_loss_epochs(self):
        return np.array(self.loss_epochs)

In [101]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error
# Load the Boston Housing dataset
adverstiment_data = 'Preprocessed/preprocessed_Adversting_data.csv'
adverstiment_data = pd.read_csv(adverstiment_data, skiprows=0)

In [102]:
adverstiment_data.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Male,Clicked on Ad
0,0.617882,0.380952,0.730472,0.916031,0,0
1,0.809621,0.285714,0.831375,0.538746,1,0
2,0.626721,0.166667,0.699200,0.797433,0,0
3,0.706272,0.238095,0.623160,0.854280,1,0
4,0.608023,0.380952,0.914568,0.731323,0,0


In [104]:
adverstiment_data.info()

adverstiment_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Daily Time Spent on Site  1000 non-null   float64
 1   Age                       1000 non-null   float64
 2   Area Income               1000 non-null   float64
 3   Daily Internet Usage      1000 non-null   float64
 4   Male                      1000 non-null   int64  
 5   Clicked on Ad             1000 non-null   int64  
dtypes: float64(4), int64(2)
memory usage: 47.0 KB


,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Male,Clicked on Ad
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
mean,0.550743,0.404976,0.626119,0.455383,0.481000,0.50000
std,0.269482,0.209180,0.204840,0.265785,0.499889,0.50025
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,0.318885,0.238095,0.504446,0.206139,0.000000,0.00000
50%,0.605388,0.380952,0.656847,0.474331,0.000000,0.50000
75%,0.781022,0.547619,0.786005,0.690232,1.000000,1.00000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


In [106]:
# Extract features (X) and target variable (y)
X = adverstiment_data.drop('Clicked on Ad', axis=1)
y = adverstiment_data['Clicked on Ad']

In [109]:
# Define neural network parameters with two hidden layers
layers = [6, 9, 15, 1]  # Input layer: 13 features, Hidden layers: 10, 8, Output layer: 1 unit
learning_rate = 0.01
momentum = 0.9
activation = 'sigmoid'
validation_percentage = 0.2
epochs = 100


In [110]:
# Create and train the neural network
nn = MyNeuralNetwork(layers, learning_rate, momentum, activation, validation_percentage)
loss_history = nn.fit(X, y, epochs)

KeyError: "None of [Index([761, 202, 727, 646, 708,  70, 600, 481, 791, 697, 750, 117, 741, 145,\n       257, 369, 194, 690, 408, 419, 733, 169, 655, 594, 458, 450, 135, 244,\n       500, 121,  74, 444],\n      dtype='int32')] are in the [columns]"